In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, KFold
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
# import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
df=pd.read_excel("Case Management Note_20200113.xlsx")

In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import LdaModel, LdaMulticore
import gensim.downloader as api
from gensim.utils import simple_preprocess, lemmatize
import re
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
logging.root.setLevel(level=logging.INFO)

In [4]:
addlist=['.',',','!','?','(',')']
stop_words = set(stopwords.words('english'))
for x in addlist:
    stop_words.add(x)
print(stop_words)

{'our', 'have', 'themselves', "shouldn't", 'ain', 'd', "that'll", 'your', 'its', 'on', 'too', 'himself', "it's", 'itself', "couldn't", ',', "weren't", 'mightn', 'yourselves', 'during', 'she', 'aren', 'did', 'are', 'not', 'don', 'because', 't', 'how', 'most', 'what', "haven't", '?', 'or', 'very', 'why', 'under', 'only', 'can', 'weren', 'doing', 'more', 'after', 'other', 'into', 'needn', 'these', 'while', 'and', 'they', 'so', 'than', 'herself', 'couldn', 'doesn', 'up', 'if', 'that', 'each', 'from', 'by', 's', 'below', 'having', 'to', 'few', 'you', 'll', 'this', 'he', 'here', 'do', 'his', 'where', 'in', 'wasn', 'off', 'for', "mightn't", 'it', 'same', 'shan', 'an', "wasn't", 'with', 'ourselves', 'yourself', '(', 're', 'won', 'has', 'being', "you'd", 'the', 'o', 'him', 'above', 'will', 'now', "you're", 'hers', 'against', "hasn't", '.', 'ours', 've', 'wouldn', 'which', 'should', 'over', 'when', "don't", 'between', 'at', 'there', "should've", "didn't", 'out', 'we', 'i', 'both', 'does', 'some'

In [5]:
def clear(x):
    out=[]
    x=str(x)
    x=x.lower()
    word_tokens=nltk.word_tokenize(x)
#    filtered_sentence = [w for w in word_tokens if not w in stop_words]
#    for i in word_tokens:
#         if i not in stop_words:
#             lemmatized_word=lemmatize(i, allowed_tags=re.compile('(NN|JJ|RB)'))
#             if lemmatized_word:
#                     out += [lemmatized_word[0].split(b'/')[0].decode('utf-8')]
#         else:
#             continue
#    word_tokens=[lemmatizer.lemmatize(x) for x in word_tokens]
    filtered_sentence = [w for w in word_tokens if not w in stop_words]  
    return filtered_sentence

df['TOKEN']=df['COMMENTS'].apply(clear)

In [6]:
df.head()

,MID,DATE_OF_NOTE,ADMIN_ENTRY,CONTACT_TYPE,CONTACT_CONNECTION,HOW_WAS_CONTACT_MADE,WHO_WAS_CONTACTED,WHY,COMMENTS,COMMENTS_CLOB,...,STAFF_WHO_PH_OTH,STAFF_WHO_PH_EXT,STAFF_WHO_PH_EXT_OTH,STAFF_WHO_PH_INT,STAFF_WHO_PH_INT_OTH,STAFF_NOTES,STAFF_REC,CMN_COMMENTS,CMN_STAFFING_TYPE,TOKEN
0,500554,09-JUN-15,No,Electronic Communications,Contact Made,Telephone,Client,"Behavior Management, Check-in, Health Services...",CW spoke to Angel today and explained that CW ...,CW spoke to Angel today and explained that CW ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CW spoke to Angel today and explained that CW ...,NaN,"[cw, spoke, angel, today, explained, cw, ran, ..."
1,500554,18-JUN-15,No,Electronic Communications,Contact Made,Telephone,"Client, Internal Collateral",Behavior Management,CW spoke to clinician Katlyn today who reports...,CW spoke to clinician Katlyn today who reports...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CW spoke to clinician Katlyn today who reports...,NaN,"[cw, spoke, clinician, katlyn, today, reports,..."
2,500554,11-MAY-15,No,Electronic Communications,Contact Made,Telephone,Internal Collateral,"Behavior Management, Check-in",CW spoke to Laura today and asked how Angel wa...,CW spoke to Laura today and asked how Angel wa...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CW spoke to Laura today and asked how Angel wa...,NaN,"[cw, spoke, laura, today, asked, angel, since,..."
3,500554,07-MAY-15,No,Program visit,Contact Made,Face-to-face,"Client, Internal Collateral","Behavior Management, Check-in, Educational/Voc...",CW met with Angel and Laura today for Angel's ...,CW met with Angel and Laura today for Angel's ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CW met with Angel and Laura today for Angel's ...,NaN,"[cw, met, angel, laura, today, angel, 's, 90, ..."
4,500554,13-MAY-15,No,Electronic Communications,Contact Made,Telephone,Client,"Behavior Management, Check-in, Educational/Voc...",CW spoke to Angel today and he reports he is a...,CW spoke to Angel today and he reports he is a...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CW spoke to Angel today and he reports he is a...,NaN,"[cw, spoke, angel, today, reports, almost, don..."


In [7]:
print(df["COMMENTS"][0])
print(df['TOKEN'][0])

CW spoke to Angel today and explained that CW ran is time time assignment concurrent with his original time assignment but will discharge 8/19 unless he can do really really well and CW will ask for an early discharge. Angel was OK with that and reports he will try his hardest.
['cw', 'spoke', 'angel', 'today', 'explained', 'cw', 'ran', 'time', 'time', 'assignment', 'concurrent', 'original', 'time', 'assignment', 'discharge', '8/19', 'unless', 'really', 'really', 'well', 'cw', 'ask', 'early', 'discharge', 'angel', 'ok', 'reports', 'try', 'hardest']


In [8]:
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [9]:
from collections import Counter

all_token=[]
for x in df['TOKEN']:
    all_token.append(x)

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out
nlp = spacy.load('en',disable=['parser', 'ner'])
lemma=lemmatization(all_token)

In [10]:
final=[]
for x in lemma:
    for y in x:
        final.append(y)
count=Counter(final)

In [11]:
print(count.most_common(200))

[('go', 89018), ('client', 88647), ('report', 83318), ('call', 80083), ('home', 73384), ('would', 61497), ('meet', 61052), ('mother', 58560), ('school', 57957), ('work', 53329), ('writer', 52126), ('speak', 51266), ('today', 49841), ('state', 49482), ('program', 49456), ('check', 45381), ('also', 44943), ('time', 42909), ('ask', 39610), ('well', 38549), ('youth', 37509), ('discuss', 36863), ('meeting', 36665), ('say', 36494), ('get', 35721), ('take', 34172), ('need', 32846), ('come', 32244), ('visit', 31501), ('inform', 31049), ('week', 30991), ('tell', 30094), ('attend', 28678), ('issue', 28168), ('day', 27704), ('know', 26734), ('see', 26471), ('mom', 26430), ('make', 26418), ('contact', 26204), ('back', 26204), ('want', 25886), ('leave', 25256), ('office', 25154), ('schedule', 24342), ('phone', 24111), ('job', 24076), ('pick', 24021), ('could', 23911), ('family', 23270), ('give', 23124), ('receive', 21988), ('pass', 21783), ('next', 20827), ('continue', 20607), ('plan', 20592), ('tr